<a href="https://colab.research.google.com/github/svafar/sys-bio/blob/main/walk_through_consensus_clustering_code_for_students.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook goes over the use of consensus clustering applied to gene expression data from a cohort of acute lymphocytic leukemia (ALL).

The code we are running this week is R, not Python. Colab also supports R. To start a new R Colab, just follow this link: [colab.to/r](https://colab.to/r). [RStudio](https://rstudio.com/) is another very convenient tool to develop R code, specially when working on your own.

# Preliminaries 


## 0.1. Install required packages
We use Bioconductor, a great tool to install packages, but there are other methods to install libraries, Bioconductor I find it the most convenient.

In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")

BiocManager::install("ALL")  ### expression data
BiocManager::install("ConsensusClusterPlus")
BiocManager::install("cluster")  ### required for silhouette function
BiocManager::install('viridis')  ### nice color palette

In [ ]:
library(ALL)
library(ConsensusClusterPlus)
library(viridis)
library(cluster)

# 1. Load data
We are going to load the ALL data.
ALL refers to Acute Lymphoblastic Leukemia and this data set has been generated by the Ritz Laboratory. The original publication is  [*Gene expression profile of adult T-cell acute lymphocytic leukemia identifies distinct subsets of patients with different response to therapy and survival*](https://pubmed.ncbi.nlm.nih.gov/14684422/)

The data consist of microarrays from 128 different individuals with acute lymphoblastic leukemia
(ALL). A number of additional covariates are available. The data have been normalized (using [RMA](https://ieeexplore.ieee.org/document/6999142))
and it is the jointly normalized data that are available here. The data are presented in the form of an
exprSet object.

In [ ]:
data(ALL)
d = exprs(ALL) # particular syntax to retrieve expression matrix values from e-Set-class

## 1.1. Get familiar with the data

We are working with 128 samples, presumably each from a patient.
For each sample, 12,625 genes have been measured.

In [ ]:
dim(d)
head(d)
d[1:5,1:5]

Let's inspect the distribution of expression for each patient. 

In [ ]:
boxplot(d[,1:50])

Remember, we are plotting the gene expression distributions per sample, ie, patient. That means that each boxplot has ~12,000 values. As you can see, expression values are very nicely distributed between 2 and 14 with some extreme values. From this distribution, we can conclude that the data we have in hands comes from microarray data and not from RNA-seq.

# Filter data

Let's filter out non-variable genes

In [ ]:
mads=apply(d,1,mad)
d=d[rev(order(mads))[1:5000],]
d[1:5,1:5]

Still, there is some variation between patients so we will scale the data to their median expression, so they are even more comparable. Other apporaches like z-score scaling are other valid options.

In [ ]:
d = sweep(d, 1, apply(d,1,median,na.rm=T)) # sweep removes, in this case substracts, the value you define, in this case the median

In [ ]:
d[1:5,1:5]

# 3. Consensus clustering

In [ ]:
maxK2test = 8
clusters = ConsensusClusterPlus(d, 
  maxK=maxK2test, # max number of K to test
  reps=50,  # number of subsample iterations
  pItem=0.8, # proportion of items (patients) to sample
  pFeature=1, # proportion of features (genes) to subsample
  clusterAlg="hc", # clustering algorithm. 
  distance="pearson", # distance between samples
  innerLinkage="complete", # hierarchical linkage method
  seed=1262118388.71279) # optional numerical value to control reproducibility

# 4. Measures to identify optimal *K*
All these measures are heuristic methods and can be difficult to unambiguously identify the number of clusters. Interpretation by the scientist is key for choosing a relevant *K*. 

## 4.1. Elbow plot.
When using [this method](https://en.wikipedia.org/wiki/Elbow_method_(clustering)) look for saturation on the amount of variance explained. The more *K*, the more variance will be explained, but at the cost of having more clusters (over-fitting).  

## 4.2. Proportion of ambiguous clustering (PAC)
Consensus index refer to the probability of two samples clustering in the same cluster. Therefore the top-right indicates elements that always cluster together. On the contrary, the bottom left are elements that never cluster together. Therefore, the middle section quantifies the elements whose cluster assignment is most ambiguous. Our objective is to limit that, so the lower the increase in that area, the better. 

In [ ]:
############## PAC implementation ##############
Kvec = 2:maxK2test
x1 = 0.1; x2 = 0.9 # threshold defining the intermediate sub-interval
PAC = rep(NA,length(Kvec)) 
names(PAC) = paste("K=",Kvec,sep="") # from 2 to maxK

for(i in Kvec){
  M = clusters[[i]]$consensusMatrix
  Fn = ecdf(M[lower.tri(M)])
  PAC[i-1] = Fn(x2) - Fn(x1)
  print(c(i, Fn(x2) - Fn(x1)))
}#end for i

# The optimal K
optK = Kvec[which.min(PAC)]
print(optK)
########################################################

# 4.3. Silhouette method
The silhouette value is a measure of how similar an object is to its own cluster (cohesion) compared to other clusters (separation). The silhouette ranges from −1 to +1, where a high value indicates that the object is well matched to its own cluster and poorly matched to neighboring clusters. If most objects have a high value, then the clustering configuration is appropriate. If many points have a low or negative value, then the clustering configuration may have too many or too few clusters.

In [ ]:
lsil = vector("list",(maxK2test-1))

for(i in 2:maxK2test){
  sil = silhouette(clusters[[i]]$consensusClass, dist(t(d), method = "euclidean"))
  sizes = table(clusters[[i]]$consensusClass)
  plot(sil, col=rep(clusters[[i]]$clrs[[3]], rep=sizes), main=paste("K=",i))
  
  sil_average = mean(sil[, 3])
  abline(v=sil_average)
  lsil[[i-1]]=sil

  print(sizes)
  print(rep(clusters[[i]]$clrs[[3]])) # this is simply to define a different color per cluster
 } 